In [61]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import scipy.optimize as spo

In [74]:
### Markowitz portfolio optimization
class Markowitz:
    """
    Returns the optimal portfolio weights and the corresponding Sharpe ratio.
    Reference: https://ocw.mit.edu/courses/18-s096-topics-in-mathematics-with-applications-in-finance-fall-2013/resources/mit18_s096f13_lecnote14/
    """
    def __init__(self, returns: list, frequency: int = 1):
        self.UNIFORM_WEIGHTS = np.ones(len(returns)) / len(returns)
        self.RETURNS = np.asarray(returns).copy()
        self.FREQUENCY = frequency
        self.COV = np.cov(self.RETURNS)
    
    def _neg_sharpe_ratio(self, weights: list, risk_free_rate: float = 0.0):
        """
        Returns the Sharpe ratio of the portfolio.
        """
        expected_return = np.dot(weights, self.RETURNS)
        standard_deviation = np.sqrt(np.dot(weights, np.dot(self.COV, weights)))*np.sqrt(self.FREQUENCY)
        return (expected_return - risk_free_rate) / standard_deviation
    
    def _variance(self, weights: list):
        """
        Returns the variance of the portfolio.
        """
        return np.dot(weights, np.dot(self.COV, weights))

    # Return Maximization
    def max_sharpe(self, risk_free_rate: float = 0.0, weight_constraint: tuple = (0,1)):
        """
        Returns the optimal portfolio weights and the corresponding Sharpe ratio.
        Use the negative Sharpe ratio as the objective function to be minimized.
        """
        # minimize the negative Sharpe ratio
        result = spo.minimize(self._neg_sharpe_ratio, self.UNIFORM_WEIGHTS, method='SLSQP', bounds=[weight_constraint] * len(self.RETURNS), constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        # weights, Sharpe ratio
        return result.x, -result.fun
        
    # Risk Minimization
    def min_variance(self, target_return: float):
        """
        Returns the optimal portfolio weights and the corresponding variance.
        Use the variance as the objective function to be minimized.
        """
        # minimize the variance
        result = spo.minimize(self._variance, self.UNIFORM_WEIGHTS, method='SLSQP', bounds=[(0,1)] * len(self.RETURNS), constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1, 'type': 'eq', 'fun': lambda x: np.dot(x, self.RETURNS) - target_return})
        # weights, variance
        return result.x, result.fun
        


In [75]:
# test max sharpe
returns = [0.1, 0.2, 0.15, 0.17, 0.12, 0.08, 0.09, 0.11, 0.13, 0.14]
markowitz= Markowitz(returns)
weights1, sharpe_ratio = markowitz.max_sharpe()
print('Weights: ', weights1)
print('Sharpe Ratio: ', sharpe_ratio)

# test min variance
weights2, variance = markowitz.min_variance(0.15)
print('Weights: ', weights2)
print('Variance: ', variance)


Weights:  [7.27581707e-12 1.25579828e-12 4.25917510e-12 3.04738240e-12
 6.07579609e-12 1.00000000e+00 3.73522047e-12 6.67552738e-12
 5.46875419e-12 4.86209859e-12]
Sharpe Ratio:  -2.147484424179913
Weights:  [0.11166098 0.12359952 0.11763025 0.12001796 0.11404869 0.10927328
 0.11046713 0.11285484 0.11524255 0.1164364 ]
Variance:  0.0001841739845625491
